### BIP 39 Reference:
* [The BIP](https://github.com/bitcoin/bips/blob/master/bip-0039.mediawiki)
* [Original Python Reference Implementation](https://github.com/trezor/python-mnemonic/blob/master/mnemonic/mnemonic.py)
* [Go](https://github.com/tyler-smith/go-bip39)
* [Javascript](https://github.com/iancoleman/jsbip39/blob/master/jsbip39.js)
* [Blog Post](https://jmcintyre.net/?p=180)
* [bitcoinlib-js](https://github.com/bitcoinjs/bip39/blob/master/index.js#L93) has the cleanest implementation.

In [33]:
%load_ext autoreload
%autoreload 2
import requests, os, binascii, hashlib, unicodedata, pbkdf2, hmac, pycoin, struct

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Grab English Wordlist From BIP Git Repo

In [34]:
english_wordlist_url = "https://raw.githubusercontent.com/bitcoin/bips/master/bip-0039/english.txt"
response = requests.get(english_wordlist_url)
wordlist = response.text.splitlines()
assert len(wordlist) == 2048

# Produce Mnemonic From Random Bytes

In [35]:
def generate_entropy(n):
    return os.urandom(n)

def mnemonic_from_entropy(entropy_bytes):
    entropy_bits_length = len(entropy_bytes) * 8
    checksum_bits_length = entropy_bits_length // 32
    sequence_length = (entropy_bits_length + checksum_bits_length) // 11
    
    # Prepare entropy bits
    entropy_bits = bin(int(binascii.hexlify(entropy), 16))[2:]
    padded_entropy_bits = entropy_bits.zfill(entropy_bits_length)
    
    # Prepare checksum
    entropy_hash = hashlib.sha256(entropy).hexdigest()
    entropy_hash_int = int(entropy_hash, 16)
    entropy_hash_bits = bin(int(entropy_hash, 16))[2:]
    padded_entropy_hash_bits = entropy_hash_bits.zfill(256)
    checksum = padded_entropy_hash_bits[:checksum_bits_length]
    
    # Take 11 bit slices of padded_entropy_bits + checksum
    # Interpret as int, pluck work from wordlist at this index
    sequence = padded_entropy_bits + checksum
    result = []
    for i in range(sequence_length):
        index = int(sequence[i * 11:(i + 1) * 11], 2)
        result.append(wordlist[index])
    return " ".join(result)

In [36]:
entropy = generate_entropy(32)
entropy

b'\xa9\n\xa4\xfa\x8d\xabu\xc9\xb3\xf3\xfc5\xaf;P\x80@CgF\x82\x83\x03\xf79\xba\xedQt\xacq)'

In [37]:
mnemonic = mnemonic_from_entropy(entropy)
mnemonic

'poverty festival dirt brave resemble tooth sound legend cup keep stage above aerobic recipe borrow expire advance right huge survey blanket file seven devote'

# Produce Seed From Mnemonic

BIP 39 uses a the PBKDF2 key-stretching algorithm to take the mnemonic seed (128, 192, or 256 bits) and a passphrase and produce a 512 bit "seed" which is then used to create BIP 32 HD Wallets.

[Here's a nice video about PBKDF2](https://www.youtube.com/watch?v=yelMxr7UErk)
* gpg, pgp, openssh applications all use it.
* pbkdf2, bcrypt, and scrypt all use this same algorithm.

Why PBKDF2?

[" PBKDF2 with 2048 rounds makes bruteforcing it 2049x harder"](https://www.reddit.com/r/Bitcoin/comments/2cm3zu/does_anyone_know_why_trezor_used_pbkdf2_in_the/cjgszrv/) and because the [recovery process](https://www.reddit.com/r/Bitcoin/comments/2cm3zu/does_anyone_know_why_trezor_used_pbkdf2_in_the/cjgtvpb/) will cause the user to leak their 24 words in random order (so brute forcing would take 24! = 620448401733239439360000 iterations)


In [38]:
def normalize_string(txt):
    if isinstance(txt, bytes):
        utxt = txt.decode('utf8')
    elif isinstance(txt, str):
        utxt = txt
    else:
        raise TypeError("String value expected")
        
    return unicodedata.normalize('NFKD', utxt)

def seed_from_mnemonic(mnemonic, passphrase=''):
    mnemonic = normalize_string(mnemonic)
    passphrase = normalize_string(passphrase)
    # FIXME: use hashlib implementation
    return pbkdf2.PBKDF2(
        mnemonic, u'mnemonic' + passphrase, 
        iterations=2048, 
        macmodule=hmac, 
        digestmodule=hashlib.sha512
    ).read(64)


In [39]:
seed = seed_from_mnemonic(mnemonic)
seed

b'Z\xfaoU\xf4*\xd2\x1e\xfc\xf3\n\x8b\xb4\r\xfeB\x82\xc5\xcb\xb8\x8dM7L\r\xf8\x06\xccy\xef\xa9\x02+U\xcd\xfd\xa01\xe1\xd1\x85\xa5\xdbD\xf9"s\xbe\xeb\xde?x]\xa1\xebn^,\xc9\xba\x1d5\xcc|'

# Master HD Wallet From Seed

In [40]:
from pycoin.key.BIP32Node import BIP32Node

In [41]:
root = BIP32Node.from_master_secret(seed)

In [42]:
root.wif()

'Kwfpd5M3HU6Jk28S5vS8eBvDyEJDtZ984XHWKfaCp1RMehEoHdBU'

In [43]:
root.child_index()

0

In [44]:
tenth_child = root.subkey(i=10)

10


In [45]:
tenth_child.child_index()

10

In [46]:
message = b"Hello, world!"

sig = root.sign(message)
sig

b'0D\x02 \x0c^\xca\x99\xa5l\x05\x8aZ\xa8\x0b\xa0\xf55\xfa_6\x83A\x9b\xaa\xd2\xd7\x1a\xcc\x13%\xcfu\x19S\x00\x02 6\xdeX\xb0S\x1b}o\xf6?\xd6\x0b\x91\xbbSR\xd2nXyf#\x7f\x1d\xd5\xd3vf\x00\xa9\xe8;'

In [47]:
root.verify(message, sig)

True

In [48]:
child_sig = tenth_child.sign(message)

In [49]:
tenth_child.verify(message, child_sig)

True

In [50]:
root.subkey(i=10).verify(message, child_sig)

True

In [51]:
BIP32Node.from_wallet_key(root.wif())

EncodingError: bad wallet key header

# Wallet Class

In [53]:
import ecdsa

class Wallet:
    
    def __init__(self, private_key):
        self.private_key = private_key
        
    def sign(self, message):
        sk = ecdsa.SigningKey.from_string(self.private_key, curve=ecdsa.SECP256k1)
        return sk.sign(message)
    
    def verify(self, signature, message):
        sk = ecdsa.SigningKey.from_string(self.private_key, curve=ecdsa.SECP256k1)
        vk = sk.get_verifying_key()
        return vk.verify(signature, message)
    
    @classmethod
    def generate(cls):
        private_key = os.urandom(32)
        return cls(private_key)

In [103]:
pk = os.urandom(32)

wallet = Wallet(pk)
sig = wallet.sign(message)
wallet.verify(sig, message)

True

# Sequential Deterministic Wallet Class

In [57]:
class SDWallet(Wallet):
    
    def child(self, n):
        private_key = self.private_key
        for _ in range(n):
            private_key = hashlib.sha256(private_key).digest()
        return SDWallet(private_key)

In [58]:
private_key = os.urandom(32)

sd1 = SDWallet(private_key)
sd2 = SDWallet(private_key)

sd1_child = sd1.child(100)
print("sd1's 100th child: ", sd1_child.private_key)

sd2_child = sd2.child(100)
print("sd2's 100th child: ", sd2_child.private_key)

assert sd1_child.private_key == sd2_child.private_key

sd1's 100th child:  b'\xba\xbb\x92\xd8S6M\n\x03\x91\xf2\xa8\x7fp\x89=Z\x08\xbdO\x8b\x9a\x87\x12=\xccjV\xc3e\xcf\xd9'
sd2's 100th child:  b'\xba\xbb\x92\xd8S6M\n\x03\x91\xf2\xa8\x7fp\x89=Z\x08\xbdO\x8b\x9a\x87\x12=\xccjV\xc3e\xcf\xd9'


In [102]:
# have one sign and the other verify

sig = sd1_child.sign(message)
sd2_child.verify(sig, message)

True

# Implementing Hierarchical Deterministic Wallet Class

In [202]:
class HDWallet(Wallet):
    
    def __init__(self, private_key, chain_code):
        super().__init__(private_key)
        self.chain_code = chain_code
        
    def child(self, i):
        # ???
        i &= 0x7fffffff
#         if is_hardened:
        i |= 0x80000000            
        exponent, chain_code = get_child(self.private_key, 
                                         self.chain_code, 
                                         i)
        private_key = exponent.to_bytes(32, 'big')
        return HDWallet(private_key, chain_code)
        

In [203]:
ORDER = ecdsa.SECP256k1.order

def get_child(private_key, chain_code_bytes, index):
    index_bytes = struct.pack(">L", index)
    secret_exponent = int.from_bytes(private_key, 'big')
    # assume hardened
    # (private key is already bytes)
    data = b"\0" + private_key + index_bytes
    I64 = hmac.HMAC(key=chain_code_bytes, msg=data, 
                    digestmod=hashlib.sha512).digest()
    I_left_as_exponent = int.from_bytes(I64[:32], 'big') % ORDER
    new_secret_exponent = (I_left_as_exponent + secret_exponent) % ORDER
    new_chain_code = I64[32:]

    return new_secret_exponent, new_chain_code

In [204]:
private_key = os.urandom(32)
chain_code = os.urandom(32)

master = private_key + chain_code

secret_exponent = int.from_bytes(private_key, 'big')

In [205]:
mine = get_child(private_key, chain_code, 0)
mine

(98482421347336734816519984682249778218199734341333331790371239960046428985489,
 b"\xcc\x1eR\x112y'\xfc\xae\x04=&\xd1N\x9eh\xd2\xf9+\xf9v\x19\xd9h..\xbe\xcfU\xc0\xc8\xb5")

In [206]:
import pycoin

theirs = pycoin.key.bip32.subkey_secret_exponent_chain_code_pair(
    secret_exponent,
    chain_code,
    0,
    True,
)

In [207]:
mine == theirs

True

In [208]:
myhd = HDWallet(private_key, chain_code)
mychild = myhd.child(5)
mychild.private_key, mychild.chain_code

(b'\xb5\xa9x\xd1\xde\x98\x1ca\x97\xaf\xf7\x9a\xa5\xe9\x16 \xdd\xdb?\x1f\xad\x1d\x11\x9b\xcbB\xa4;\xdc\xe6\xcex',
 b'\xd9 :\x90\xda\x07\xf2=u\x90\x98#fF\r\xc4\xb5\x8d\xf0\xce\xea3\x10\x03.DA\xee\xe9\xd1c\xa1')

In [209]:
from pycoin.key.BIP32Node import BIP32Node

theirhd = BIP32Node(netcode=0, chain_code=chain_code, secret_exponent=secret_exponent)

In [210]:
theirchild = theirhd.subkey(5, is_hardened=True, as_private=True)
theirchild.chain_code()

b'\xd9 :\x90\xda\x07\xf2=u\x90\x98#fF\r\xc4\xb5\x8d\xf0\xce\xea3\x10\x03.DA\xee\xe9\xd1c\xa1'

In [211]:
int.from_bytes(mychild.private_key, 'big') == theirchild.secret_exponent()

True

In [212]:
mychild.chain_code == theirchild.chain_code()

True